# Symbolic Regression vs. Neural Networks on Multiscale L96

To recap from the previous notebooks, [Lorenz (1996)](https://www.ecmwf.int/en/elibrary/10829-predictability-problem-partly-solved) describes a "two time-scale" model in two equations which are:
\begin{align}
\frac{d}{dt} X_k
&= - X_{k-1} \left( X_{k-2} - X_{k+1} \right) - X_k + F - \left( \frac{hc}{b} \right) \sum_{j=0}^{J-1} Y_{j,k}
\\
\frac{d}{dt} Y_{j,k}
&= - cbY_{j+1,k} \left( Y_{j+2,k} - Y_{j-1,k} \right) - c Y_{j,k} + \frac{hc}{b} X_k
\end{align}

This model can be visualized as follows:

```{figure} https://www.researchgate.net/publication/319201436/figure/fig1/AS:869115023589376@1584224577926/Visualisation-of-a-two-scale-Lorenz-96-system-with-J-8-and-K-6-Global-scale-values.png
:width: 400px
:name: l96-equation-figure-symbolicvnn

*Visualisation of a two-scale Lorenz '96 system with J = 8 and K = 6. Global-scale variables ($X_k$) are updated based on neighbouring variables and on the local-scale variables ($Y_{j,k}$) associated with the corresponding global-scale variable. Local-scale variabless are updated based on neighbouring variables and the associated global-scale variable. The neighbourhood topology of both local and global-scale variables is circular. Image from [Exploiting the chaotic behaviour of atmospheric models with reconfigurable architectures - Scientific Figure on ResearchGate.](https://www.researchgate.net/figure/Visualisation-of-a-two-scale-Lorenz-96-system-with-J-8-and-K-6-Global-scale-values_fig1_319201436)*.
```

In [ ]:
import os
import time
from IPython.display import display, SVG

import gplearn
import gplearn.genetic
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydot
import pysindy as ps
import torch
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

from L96_model import L96

In [ ]:
# Ensuring reproducibility
np.random.seed(14)
torch.manual_seed(14);

## Dataset Generation

Let's generate some data using the same conditions as in the previous notebooks [Introduction to Neural Networks](https://m2lines.github.io/L96_demo/notebooks/Universal_approximation.html) and [Using Neural Networks for L96 Parameterization](https://m2lines.github.io/L96_demo/notebooks/Neural_network_for_Lorenz96.html).

### Obtaining the Ground Truth

In [ ]:
time_steps = 20000
forcing, dt, T = 18, 0.01, 0.01 * time_steps

W = L96(8, 32, F=forcing)

X_true, _, _, subgrid_tend = W.run(dt, T, store=True, return_coupling=True)
X_true, subgrid_tend = X_true.astype(np.float32), subgrid_tend.astype(np.float32)

print(f"X_true shape: {X_true.shape}")
print(f"subgrid_tend shape: {subgrid_tend.shape}")

In this case, although we have 8 different dimensions in $X$, each is derived in the exact same way from neighboring values of $X$ (and local subdimensions $Y$). Therefore, we can actually reduce this problem from an 8-to-8D regression problem down to an 8-to-1D regression problem:

In [ ]:
Dx = X_true.shape[1]
inputs = np.vstack([X_true[:, range(-i, Dx - i)] for i in range(Dx)])
targets = np.hstack([subgrid_tend[:, -i] for i in range(Dx)])

print(f"inputs shape: {inputs.shape}")
print(f"targets shape: {targets.shape}")

In [ ]:
plt.figure(figsize=(16, 4), dpi=150)
plt.title("Multiscale L96 trajectory", fontsize=16)
for i in range(Dx):
    plt.plot(inputs[:, i][:500], label=r"$x" + str(i) + "$")
plt.legend(ncol=2, fontsize=14)
plt.xlabel("Time", fontsize=16)
plt.ylabel("Value", fontsize=16)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=150)
fig.suptitle("2D histograms of features vs. forcing", fontsize=16)
for i in range(Dx):
    plt.subplot(2, 4, i + 1)
    plt.hist2d(inputs[:, i], targets, bins=50)
    plt.xlabel(f"$x_{i}$", fontsize=16)
    plt.xticks(np.arange(-6, 13, 2))
    plt.xlim(-8, 14)
    if i % 4 == 0:
        plt.ylabel("Forcing", fontsize=16)
plt.tight_layout()

Looking at these marginal histograms of each large-scale variable vs. the subgrid forcing, we can see there's a mostly linear relationship between $x_0$ and its forcing, suggesting that the subgrid terms tend to force small values higher and large values lower. This suggests that an extremely simple linear model only containing $x_0$ with negative slope will do reasonably well. However, the relationship isn't _completely_ linear, and also contains a bifurcation at larger $x_0$.

For the other features, the relationships are less straightforward, but there's clearly high mutual information.

### Split the Data into Training and Test Sets

In [ ]:
data_idxs = np.arange(len(inputs))
np.random.shuffle(data_idxs)

num_train_samples = int(len(inputs) * 0.75)
train_idxs = data_idxs[:num_train_samples]
test_idxs = data_idxs[num_train_samples:]

### Build the Dataset and the Dataloaders

In [ ]:
BATCH_SIZE = 1024

# Training Data
train_dataset = TensorDataset(
    torch.tensor(inputs[train_idxs]), torch.tensor(targets[train_idxs])
)
loader_train = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Test Data
test_dataset = TensorDataset(
    torch.tensor(inputs[test_idxs]), torch.tensor(targets[test_idxs])
)
loader_test = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Creating and Training the Neural Network

Now let's create a neural network and train it on the dataset we generated above.

### Define Functions to Train and Evaluate the Model

```{note}
The code in this section is taken from the notebook [Introduction to Neural Networks](https://m2lines.github.io/L96_demo/notebooks/Universal_approximation.html).
```

In [ ]:
def train_model(network, criterion, loader, optimizer):
    """Train the network for one epoch"""
    network.train()

    train_loss = 0
    for batch_x, batch_y in loader:
        # Get predictions
        if len(batch_x.shape) == 1:
            # This if block is needed to add a dummy dimension if our inputs are 1D
            # (where each number is a different sample)
            prediction = torch.squeeze(network(torch.unsqueeze(batch_x, 1)))
        else:
            prediction = network(batch_x)

        # Compute the loss
        loss = criterion(prediction, batch_y)
        train_loss += loss.item()

        # Clear the gradients
        optimizer.zero_grad()

        # Backpropagation to compute the gradients and update the weights
        loss.backward()
        optimizer.step()

    return train_loss / len(loader)


def test_model(network, criterion, loader):
    """Test the network"""
    network.eval()  # Evaluation mode (important when having dropout layers)

    test_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in loader:
            # Get predictions
            if len(batch_x.shape) == 1:
                # This if block is needed to add a dummy dimension if our inputs are 1D
                # (where each number is a different sample)
                prediction = torch.squeeze(network(torch.unsqueeze(batch_x, 1)))
            else:
                prediction = network(batch_x)

            # Compute the loss
            loss = criterion(prediction, batch_y)
            test_loss += loss.item()

        # Get an average loss for the entire dataset
        test_loss /= len(loader)

    return test_loss


def fit_model(network, criterion, optimizer, train_loader, val_loader, n_epochs):
    """Train and validate the network"""
    train_losses, val_losses = [], []
    start_time = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_model(network, criterion, train_loader, optimizer)
        val_loss = test_model(network, criterion, val_loader)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
    end_time = time.time()
    print(f"Training completed in {int(end_time - start_time)} seconds.")

    return train_losses, val_losses

### Create the Neural Network

In [ ]:
class Net_ANN(nn.Module):
    def __init__(self):
        super(Net_ANN, self).__init__()
        self.linear1 = nn.Linear(8, 16)
        self.linear2 = nn.Linear(16, 16)
        self.linear3 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)[:, 0]
        return x

### Train the Network

In [ ]:
nn_3l = Net_ANN()

n_epochs = 20
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(nn_3l.parameters(), lr=0.01)
train_loss, validation_loss = fit_model(
    nn_3l, criterion, optimizer, loader_train, loader_test, n_epochs
)

### Visualize Results

In [ ]:
plt.figure(dpi=150)
plt.plot(train_loss, "b", label="Training loss")
plt.plot(validation_loss, "r", label="Validation loss")
plt.legend();

In [ ]:
preds_nn = nn_3l(torch.from_numpy(inputs[test_idxs])).detach().numpy()
print("R2 Score:", r2_score(preds_nn, targets[test_idxs]))

The neural network gets an $R^2$ of ~ 0.916, which seems pretty good given the non-determinism of the problem.

## Comparing with Linear Regression

In [ ]:
linear_reg = LinearRegression()
linear_reg.fit(inputs[train_idxs], targets[train_idxs])
print("R2 Score:", linear_reg.score(inputs[test_idxs], targets[test_idxs]))

Linear regression also does fairly well, though about 0.1 lower in terms of $R^2$. Lets look at its coefficients:

In [ ]:
plt.figure(dpi=150)
plt.bar(range(8), linear_reg.coef_)
plt.xticks(range(8), [f"x{i}" for i in range(8)], fontsize=12)
plt.ylabel("Coefficient value", fontsize=12)
plt.show()

This matches the 2D histogram plots from earlier that showed a negative linear relationship between forcing and $x_0$.

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.title("Neural Network vs. Linear Regression", fontsize=16)

preds_nn = nn_3l(torch.from_numpy(inputs)).detach().numpy()
preds_lr = linear_reg.predict(inputs)
random_idx = np.random.choice(len(targets - 1000))
random_slice = slice(random_idx, random_idx + 1000)

plt.plot(preds_nn[random_slice], label="NN Predicted values")
plt.plot(preds_lr[random_slice], label="LR Predicted values")
plt.plot(
    targets[random_slice], label="True values", ls="--", color="black", alpha=0.667
)
plt.xlabel(f"Time (starting at idx {random_idx})", fontsize=16)
plt.ylabel("Subgrid forcing", fontsize=16)
plt.legend(fontsize=14)
plt.show()

## Symbolic regression with genetic programming

Let's try using one of the genetic programming libraries to discover a parameterization:

In [ ]:
symbolic_reg = gplearn.genetic.SymbolicRegressor(
    population_size=5000,
    generations=25,
    p_crossover=0.7,
    p_subtree_mutation=0.1,
    p_hoist_mutation=0.05,
    p_point_mutation=0.1,
    max_samples=0.9,
    verbose=1,
    parsimony_coefficient=0.001,
    const_range=(-2, 2),
    function_set=("add", "mul", "sub", "div", "sin", "log"),
)

symbolic_reg.fit(inputs[train_idxs][:20000], targets[train_idxs][:20000])

In [ ]:
print(symbolic_reg._program)

In [ ]:
print(
    "R2 Score:", r2_score(symbolic_reg.predict(inputs[test_idxs]), targets[test_idxs])
)

While the program does slightly better than linear regression, it's extremely complicated (graph visualization below for slightly easier inspection):

In [ ]:
# Build the graph
graph = pydot.graph_from_dot_data(symbolic_reg._program.export_graphviz())[0]
graph.write_svg("figs/symbolic_regression.svg")

# Display the graph
display(SVG("figs/symbolic_regression.svg"))

### Explore the Pareto frontier

We can explore the Pareto frontier of the final population of programs as well, to see if there's a simpler program that still does well

In [ ]:
df = pd.DataFrame.from_dict(
    [
        dict(
            idx=i,
            expr=str(p),
            fitness=p.fitness_,
            length=p.length_,
            r2_score=r2_score(p.execute(inputs[test_idxs]), targets[test_idxs]),
        )
        for i, p in enumerate(symbolic_reg._programs[-1])
    ]
)

In [ ]:
pareto_frontier = []
used_programs = set()

for l in range(1, 40):
    programs = df[df.length <= l]
    best = programs.r2_score.argmax()
    idx = programs.idx.values[best]
    r2 = programs.r2_score.values[best]
    program = symbolic_reg._programs[-1][idx]
    if program not in used_programs:
        pareto_frontier.append(dict(length=l, r2=r2, expr=str(program)))
        used_programs.add(program)

pd.set_option("max_colwidth", 200)
pareto_frontier = pd.DataFrame.from_dict(pareto_frontier)

In [ ]:
pareto_frontier

Unfortunately, nothing really jumps out here as both simple and performant.

## Symbolic Regression with Sparse Linear Regression and Polynomial Features

We can also apply sparse regression methods to this problem with polynomial features

In [ ]:
def learned_expr(coef, names, cutoff=1e-3):
    coef_idx = np.argwhere(np.abs(coef) > cutoff)[:, 0]
    sort_order = reversed(np.argsort(np.abs(coef[coef_idx])))
    return " + ".join(
        [
            f"{coef[coef_idx[i]]:.4f}*{names[coef_idx[i]].replace(' ','*')}"
            for i in sort_order
        ]
    )

In [ ]:
pf = PolynomialFeatures(2)
feats = pf.fit_transform(inputs)
names = pf.get_feature_names_out([f"x_{i}" for i in range(8)])

### Non-sparse linear regression

In [ ]:
linear_reg = LinearRegression()
linear_reg.fit(feats[train_idxs], targets[train_idxs])
learned_expr(linear_reg.coef_, names)

In [ ]:
print("R2 Score:", linear_reg.score(feats[test_idxs], targets[test_idxs]))

Adding polynomial features brings our $R^2$ much closer to that of the neural network, but we have a ton of terms. Let's try making the regression sparser.

### Sequentially thresholded least squares

In [ ]:
stlsq = ps.optimizers.stlsq.STLSQ(alpha=0.001, threshold=0.01)
stlsq.fit(feats[train_idxs], targets[train_idxs])
learned_expr(stlsq.coef_[0], names)

In [ ]:
stlsq.score(feats[test_idxs], targets[test_idxs])

Looks like we can get a much sparser expression without sacrificing too much performance, though if we increase the threshold slightly, we will see a trade-off:

In [ ]:
stlsq2 = ps.optimizers.stlsq.STLSQ(alpha=0.001, threshold=0.02)
stlsq2.fit(feats[train_idxs], targets[train_idxs])
learned_expr(stlsq2.coef_[0], names)

In [ ]:
stlsq2.score(feats[test_idxs], targets[test_idxs])

This is much simpler, but less performant. It still strongly outperformed the particluar genetic programming method we used, though. Let's visualize some of the predictions:

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.title("Neural network vs. quadratic regressions", fontsize=16)

preds_nn = nn_3l(torch.from_numpy(inputs)).detach().numpy()
preds_lr_quad = linear_reg.predict(feats)
preds_st_quad = stlsq.predict(feats)
preds_st_quad2 = stlsq2.predict(feats)
random_idx = np.random.choice(len(targets - 1000))
random_slice = slice(random_idx, random_idx + 1000)

plt.plot(preds_nn[random_slice], label="NN predicted values")
plt.plot(preds_lr_quad[random_slice], label="LR + quadratic")
plt.plot(preds_st_quad[random_slice], label=r"STLSQ + quadratic, threshold=$0.01$")
plt.plot(preds_st_quad2[random_slice], label=r"STLSQ + quadratic, threshold=$0.02$")
plt.plot(
    targets[random_slice], label="True values", ls="--", color="black", alpha=0.667
)
plt.xlabel(f"Time (starting at idx {random_idx})", fontsize=16)
plt.ylabel("Subgrid forcing", fontsize=16)
plt.legend(fontsize=14)
plt.show()

## Add threshold features

The 2D histogram of $x_0$ looks like it might be potentially modelable by piecewise linear regression:

In [ ]:
plt.figure(figsize=(4, 4), dpi=150)
plt.hist2d(inputs[:, 0], targets, bins=50)
plt.xlim(-6, 14)
plt.xlabel("$x_0$", fontsize=16)
plt.ylabel("Forcing", fontsize=16)
plt.show()

### Sparse regression

Let's see if sparse regression will discover that if we provide it with thresholding features and cross-terms:

In [ ]:
x0_thresholds = np.array(
    [np.ones_like(inputs[:, 0])] + [inputs[:, 0] > i for i in range(6)]
)
x0_threshold_names = [""] + [f"*(x0>{t})" for t in range(6)]

feats2 = np.array(
    [
        feats[:, i] * x0_thresholds[j]
        for i in range(feats.shape[1])
        for j in range(len(x0_thresholds))
    ]
).T
names2 = [
    f"{names[i]}{x0_threshold_names[j]}"
    for i in range(feats.shape[1])
    for j in range(len(x0_thresholds))
]

In [ ]:
stlsq3 = ps.optimizers.stlsq.STLSQ(alpha=0.001, threshold=0.25)
stlsq3.fit(feats2[train_idxs], targets[train_idxs])
learned_expr(stlsq3.coef_[0], names2)

In [ ]:
stlsq3.score(feats2[test_idxs], targets[test_idxs])

In [ ]:
plt.figure(figsize=(4, 4), dpi=150)
plt.hist2d(inputs[:, 0], targets, bins=50)

fn = (
    lambda x0: -2.1200 * 1 * (x0 > 2)
    + -1.2695 * 1 * (x0 > 0)
    + -0.9488 * x0
    + -0.6694 * 1 * (x0 > 4)
    + 0.5175 * x0 * (x0 > 1)
)

xt = np.linspace(-6, 14, 100)
yt = fn(xt)
plt.plot(xt, yt, color="red")
plt.xlim(-6, 14)
plt.xlabel("$x_0$", fontsize=16)
plt.ylabel("Forcing", fontsize=16)
plt.show()

It looks like the model learns a piecewise linear function in a single variable, even though we didn't explicitly require it!

### Genetic programming

We can also try using genetic programming here, but with a different function set that removes some of the nonlinear operators we included before and replaces them with min and max, which allow for piecewise behavior:

In [ ]:
symbolic_reg_pw = gplearn.genetic.SymbolicRegressor(
    population_size=5000,
    generations=50,
    p_crossover=0.7,
    p_subtree_mutation=0.1,
    p_hoist_mutation=0.05,
    p_point_mutation=0.1,
    max_samples=0.9,
    verbose=1,
    parsimony_coefficient=0.001,
    const_range=(-2, 2),
    function_set=("add", "mul", "max", "min"),
)
symbolic_reg_pw.fit(inputs[train_idxs][:20000], targets[train_idxs][:20000])

In [ ]:
print(symbolic_reg_pw._program)

In [ ]:
print(
    "R2 Score:",
    r2_score(symbolic_reg_pw.predict(inputs[test_idxs]), targets[test_idxs]),
)

While this leads to slightly higher $R^2$ than before, the model is still pretty complex:

In [ ]:
# Build the graph
graph = pydot.graph_from_dot_data(symbolic_reg_pw._program.export_graphviz())[0]
graph.write_svg("figs/symbolic_regression_pw.svg")

# Display the graph
from IPython.display import Image, display

display(SVG("figs/symbolic_regression_pw.svg"))

Tuning the hyperparameters could help, but requires more compute.